# Model calibration using the trajectory

This is the module that using a predefined "trajectory" to tune 
parameters of mixed-in effect

<style>
    body {
        --vscode-font-family: "Lato"
    }
    h1, h2, h3 {
        
    }
</style>
<script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
<script src="https://cdn.plot.ly/plotly-2.6.3.min.js"></script>

## Initialization

In [1]:
ENV["CRIMINOS_CONF"] = "confs/conf_single_nij.yaml"
ENV["CRIMINOS_ALIAS"] = "test"
Base.istextmime(::MIME"application/vnd.plotly.v1+json") = true
using Revise
using ForwardDiff
using LinearAlgebra
using Random
using Printf
using LaTeXStrings
using JuMP
using Criminos
using Plots
using Gurobi
using ProgressMeter
using ColorSchemes

using CSV, Tables, DataFrames, YAML

include("./conf.jl")
include("./tools.jl")

# yaml = YAML.load_file("ipynb/nij-simulation.0.15.yaml")
# yaml = yaml2 = YAML.load_file("ipynb/nij-simulation.0.15.yaml")
yaml = yaml2 = YAML.load_file("ipynb/nij-simulation.yaml")


# maximum number of time periods used in the estimation
Tₘ = 50

data_fixed = AgentData(yaml, Tₘ)
data_simulation = AgentData(yaml2, Tₘ; n=data_fixed.n)

Cardinal Optimizer v7.2.4. Build date Dec  6 2024
Copyright Cardinal Operations 2024. All Rights Reserved


ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


Cardinal Optimizer v7.2.4. Build date Dec  6 2024
Copyright Cardinal Operations 2024. All Rights Reserved


┌ Warning: Replacing docs for `Criminos.decision_matching_lh_opt_with_threshold! :: Union{Tuple{Tm}, Tuple{Tx}, Tuple{R}, Tuple{Array{Criminos.MarkovState{R, Tx}, 1}, Array{Criminos.BidiagSys{Tx, Tm}, 1}}} where {R, Tx, Tm}` in module `Criminos`
└ @ Base.Docs docs/Docs.jl:243


--------------------------------------------------------------------------------
config_dir: confs/conf_single_nij.yaml
result_dir: result-20250313/test
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Simulation Style Configs:

--------------------------------------------------------------------------------
total number of slots: 18


OrderedCollections.OrderedDict{Any, Any} with 23 entries:
  "R"                            => 1
  "bool_compute"                 => false
  "bool_conv"                    => true
  "bool_init"                    => true
  "bool_plot_surface"            => false
  "bool_plot_trajectory"         => false
  "bool_use_html"                => true
  "group_montonicity"            => [0, 0]
  "group_new_ratio"              => [1, 4]
  "group_size"                   => [1, 6]
  "output_dir"                   => "result-20250313/test"
  "seed_number"                  => 1
  "style_correlation"            => "nothing"
  "style_correlation_psd"        => true
  "style_correlation_seed"       => "rand"
  "style_correlation_subp"       => false
  "style_decision"               => "decision_identity!"
  "style_mixin"                  => "mixed_in_gnep_best!"
  "style_mixin_monotonicity"     => 2
  ⋮                              => ⋮

┌ Info: size corrected to
│   this.n = 17
└ @ Main /Users/brent/Documents/recherche/23-fairness/Criminos.jl/tools.jl:198
┌ Info: size corrected to
│   this.n = 17
└ @ Main /Users/brent/Documents/recherche/23-fairness/Criminos.jl/tools.jl:222


AgentData{Vector{Float64}, Matrix{Float64}}(17, 50, [0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781, 0.781], [230.0, 213.0, 308.0, 238.0, 255.0, 226.0, 228.0, 230.0, 216.0, 195.0, 187.0, 179.0, 140.0, 102.0, 44.0, 13.0, 1.0], [1382.0 1114.3 … 527.1 525.9; 1278.0 1223.8 … 734.8 733.7; … ; 77.0 130.9 … 237.4 235.6; 8.0 34.3 … 123.4 123.2], [1382.0 1056.344068465773 … 504.03899395082686 489.31844727188303; 1278.0 1152.3798347803647 … 693.9338771107412 664.647194119283; … ; 77.0 103.38982006601921 … 199.07037699115736 199.2219846610621; 8.0 22.0733896766111 … 87.07149879227056 87.86106962569467], [1382.0 1172.2559315342269 … 550.1610060491731 562.4815527281169; 1278.0 1295.2201652196352 … 775.6661228892588 802.7528058807171; … ; 77.0 158.4101799339808 … 275.7296230088427 271.9780153389379; 8.0 46.526610323388894 … 159.72850120772944 158.53893037430532], [257.4 278.1 … 144.3 147.6; 247.9 308.8 … 211.3 210.1; … ; 39.1 46.0 … 67.

In [2]:
data_simulation.γ

17-element Vector{Float64}:
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781
 0.781

## Start

> agent-based simulation results

In [3]:
n = data_fixed.n

17

In [4]:
data_fixed.traj_x

17×50 Matrix{Float64}:
 1382.0  1114.3   883.6   730.5   634.6  …   522.5   526.3   527.1   525.9
 1278.0  1223.8  1149.7  1053.2   971.2      736.5   736.5   734.8   733.7
 1849.0  1619.4  1491.4  1387.3  1322.9     1067.3  1068.7  1065.7  1062.0
 1429.0  1388.3  1333.1  1310.2  1270.3     1078.2  1075.5  1069.1  1069.3
 1530.0  1406.6  1350.3  1305.9  1262.8     1117.1  1119.0  1115.4  1122.2
 1358.0  1298.2  1249.2  1219.9  1198.9  …  1078.0  1071.9  1070.1  1074.5
 1368.0  1277.2  1231.0  1184.7  1154.3     1074.8  1070.0  1069.2  1062.5
 1377.0  1283.2  1213.0  1174.7  1144.4     1048.6  1047.6  1058.4  1059.1
 1294.0  1230.7  1176.5  1147.8  1116.0     1019.2  1021.3  1019.1  1012.1
 1170.0  1124.4  1093.0  1058.6  1051.2      938.8   941.9   955.2   963.8
 1122.0  1087.4  1055.6  1026.2   998.4  …   909.9   910.3   896.4   902.8
 1074.0  1008.3   985.8   972.1   945.4      875.4   868.7   865.1   861.3
  838.0   866.6   850.4   838.6   832.6      760.5   761.4   762.0   764.2
  

### Initialization and override the parameters of $\Psi$

In [5]:
data = data_fixed
include("./init.jl")
# baseline policy
for _z in vec_z
    _z.τ .= 0.3
end
vec_Ψ[1].γ = data.γ
vec_Ψ[1].λ = data.λ
# vec_Ψ[1].λ[1] = 497.26
vec_Ψ[1].Γ = diagm(data.γ)
vec_Ψ[1].Γₕ = vec_Ψ[1].M * vec_Ψ[1].Γ
vec_z[1].x .= data.traj_x[:, 1]
vec_z[1].x₋ .= data.traj_x[:, 1]
vec_z[1].y = data.traj_y[:, 1]

17-element Vector{Float64}:
 257.4
 247.9
 429.1
 377.5
 428.6
 401.5
 420.3
 444.5
 444.3
 416.5
 383.8
 401.0
 308.5
 245.3
 113.3
  39.1
   3.3

In [6]:
Criminos.default_barrier_option.μ

0.05

## Fitting

In [7]:
_args = (ω∇ω, G, md, _H, _g) = fit_trajectory(data, vec_Ψ)

Setting parameter 'RelGap' to 0.01
Setting parameter 'FeasTol' to 0.0001
Setting parameter 'DualTol' to 0.0001
Model fingerprint: 169c5775

Using Cardinal Optimizer v7.2.4 on macOS (aarch64)
Hardware has 14 cores and 14 threads. Using instruction set ARMV8 (30)
Maximizing an SDP problem

The original problem has:
    2516 rows, 6817 columns and 9996 non-zero elements
    2 PSD columns
    1 PSD objective elements
    153 PSD constraints

Using Primal-Dual Method

Starting barrier solver using 14 threads

Problem info:
Range of matrix coefficients:    [6e-03,1e+00]
Range of rhs coefficients:       [2e-02,2e-02]
Range of bound coefficients:     [0e+00,0e+00]
Range of cost coefficients:      [2e-04,2e+03]

Factor info:
Number of dense columns:         1
Number of matrix entries:        3.308e+04
Number of factor entries:        3.846e+04
Number of factor flops:          1.842e+06

Iter       Primal.Obj         Dual.Obj      Compl  Primal.Inf  Dual.Inf    Time
   0  -2.61911932e+02  +0.000

UndefVarError: UndefVarError: `_args` not defined

In [8]:
_H, _g

UndefVarError: UndefVarError: `_H` not defined

## Using fitted params at fixed control $\tau$

In [9]:
Fp(vec_z) = F!(
    vec_z, vec_Ψ;
    fₜ=Criminos.decision_identity!, targs=(cₜ, cc.τₗ, cc.τₕ),
    fₘ=cc.style_mixin, margs=_args,
)
kₑ, ε, traj, bool_opt = Criminos.simulate(
    vec_z, vec_Ψ, Fp; K=K,
    metrics=metrics,
    tol=1e-7
);

UndefVarError: UndefVarError: `K` not defined

In [10]:
plot_convergence(ε, vec_z |> length)
# the equilibria
r = traj[end][1]

UndefVarError: UndefVarError: `ε` not defined

#### Visualize

In [11]:
figs_vis = plot_fitting_results(data, r; bool_pdf=false);

UndefVarError: UndefVarError: `plot_fitting_results` not defined

In [12]:
plot(figs_vis[1], figs_vis[2])

UndefVarError: UndefVarError: `figs_vis` not defined

In [13]:
plot(figs_vis[3], figs_vis[4])

UndefVarError: UndefVarError: `figs_vis` not defined

In [14]:
# savefig(fig, "nij-simulation-y.pdf")
# savefig(fig1, "nij-simulation-x.pdf")
# savefig(fig2, "nij-simulation-y-prog.pdf")
# savefig(fig3, "nij-simulation-x-prog.pdf")

## Dynamic control 

In [15]:
Fp(vec_z) = F!(
    vec_z, vec_Ψ;
    fₜ=Criminos.decision_priority_by_opt!, targs=(cₜ, cc.τₗ, cc.τₕ),
    fₘ=cc.style_mixin, margs=_args,
)
kₑ, ε, traj, bool_opt = Criminos.simulate(
    vec_z, vec_Ψ, Fp; K=K,
    metrics=metrics,
    tol=1e-7
);

UndefVarError: UndefVarError: `K` not defined

In [16]:
@info """sanity check 
$(abs.(Criminos.decision_priority!(traj[end][1], vec_Ψ[1]; args=(cₜ, cc.τₗ, cc.τₕ)) - traj[end][1].I) |> maximum)
"""

UndefVarError: UndefVarError: `traj` not defined

In [17]:
fig = generate_empty(cc.bool_use_html)
plot!(
    1:n, traj[end][1].y,
    labels=L"$\rho_T$",
    linestyle=:dot,
    color=series_color[1],
    linewidth=3,
)
plot!(
    1:n, data.traj_y_lb[:, end], fillrange=data.traj_y_ub[:, end],
    labels=L"$\bar \rho_T",
    fillalpha=0.3,
    linestyle=:dot,
    color=series_color[1],
)
fig

UndefVarError: UndefVarError: `cc` not defined

In [18]:
cc

UndefVarError: UndefVarError: `cc` not defined

In [19]:
[traj[end][1].τ r.τ]

UndefVarError: UndefVarError: `traj` not defined